In [3]:
from numpy.typing import NDArray
from typing import Any
from fianco import *

import numpy as np

def shift_bits(x: int, y: int, num_bits: int, start_pos: int) -> NDArray[Any]:
    # Create a mask to extract the bits from x
    mask = ((1 << num_bits) - 1) << start_pos
    
    # Extract the bits from x
    bits_to_shift = (x & mask) >> start_pos
    
    # Clear the corresponding bits in y
    y &= ~(((1 << num_bits) - 1) << start_pos)
    
    # Combine the extracted bits with y
    y |= bits_to_shift << start_pos
    
    return y

# Example usage
x = 5
y = 0b0

# Shift 4 bits from x to y, starting at position 4
result = shift_bits(x, y, 4, 0)

print(f"x: {bin(x)}")
print(f"y: {bin(y)}")
print(f"Result: {bin(result)}")


x: 0b101
y: 0b0
Result: 0b101


In [4]:
LOCK = 0
MOVE = 1
SCORE = 2
FLAG = 3
HEIGHT = 4


def make_tt(size: int, rnd: np.random.Generator) -> tuple[Any, Any]:
    tt_lck = np.zeros((size,), dtype=np.uint64) # hash key
    tt_mov = np.zeros((size, 2, 2), dtype=np.uint8) # (type) number
    tt_scr = np.zeros((size,), dtype=np.float16) # score
    tt_flg = np.zeros((size,), dtype=np.uint8) # type of score
    tt_hgt = np.zeros((size,), dtype=np.uint64) # depth recorded
    tt_wrt = np.zeros((size,), dtype=np.bool_) # who moved?
    r_mat = rnd.integers(0, np.iinfo(tt_lck.dtype).max, (9, 9, 2)) # random integers
    tt = (tt_lck, tt_mov, tt_scr, tt_flg, tt_hgt, tt_wrt)
    return r_mat, tt


def enc_brd(brd: Mat, r_mat: NDArray[Any], tt: tuple[NDArray[Any], ...]) -> int:
    hsh = 0
    for y in range(brd.shape[0]):
        for x in range(brd.shape[1]):
            if np.any(brd[:, y, x]):
                wrt = brd[0, y, x]
                hsh ^= r_mat[y, x, wrt]


def rtt(idx: int, tt: TT) -> NDArray[Any]:
    ...


def wtt(idx: int, dta: NDArray[Any], tt: TT) -> bool:
    ...

NameError: name 'TT' is not defined

In [30]:
from numpy import iinfo

iinfo(np.uint32).max

4294967295

In [10]:
from numba.experimental import jitclass

import numba as nb

@jitclass([('in_root', nb.bool),
           ('for_player', nb.uint8),
           ('end_state', nb.int8),
           ('board_state', nb.int8),
           ('move_from', nb.uint8),
           ('move_to', nb.uint8),
           ('depth', nb.uint16),
           ('alpha', nb.int16),
           ('beta', nb.int16)]) # type: ignore
class SearchState:
    in_root: bool
    
    for_player: int
    
    end_state: Mat
    board_state: Mat
    
    move_from: NDArray[number]
    move_to: NDArray[number]

    depth: float
    alpha: float
    beta: float

    def __init__(self) -> None:
        self.in_root = True
        self.for_player = 1
        self.end_state = np.empty((2, 9, 9), dtype=np.int16) # type: ignore
        self.board_state = np.empty((2, 9, 9), dtype=np.int16) # type: ignore
        self.move_from = np.empty((2, 9, 9), dtype=np.uint16) # type: ignore
        self.move_to = np.empty((2, 9, 9), dtype=np.uint16) # type: ignore
        self.depth = 0
        self.alpha = 0
        self.beta = 0

sst = SearchState()